# Digital EMG Signal Feature Extraction

Import packages.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from filter import BandpassFilter1D, NotchFilter1D
from processing import MeanShift1D, Detrend1D, Resample1D, Normalize1D
from segment import SegmentND
from feature_extraction import *
%matplotlib inline

EMG signal processing functions

In [2]:
# process signal of each channel
def process_signal1d(x, raw_fs=1000, low_fs=1, high_fs=120, notch_fs=60, Q=20, window_size=250, step_size=50, target_fs=512):
    """
    @param x: signal of a single channel
    @param raw_fs: original sampling rate
    @param low_fs: low cutoff frequency
    @param high_fs: high cutoff frequency
    @param notch_fs: notch cutoff frequency
    @param Q: Q factor
    @param window_size: windows size for detrending
    @param step_size: step size for detrending
    @param target_fs: target sampling rate for resampling step
    """
    # mean-correct signal
    x_processed = MeanShift1D.apply(x)
    # filtering noise
    x_processed = BandpassFilter1D.apply(x_processed, low_fs, high_fs, order=4, fs=raw_fs)
    x_processed = NotchFilter1D.apply(x_processed, notch_fs, Q=Q, fs=raw_fs)
    # detrend
    x_processed = Detrend1D.apply(x_processed, detrend_type='locreg', window_size=window_size, step_size=step_size)
    # resample
    x_processed = Resample1D.apply(x_processed, raw_fs, target_fs)
    # rectify
    x_processed = abs(x_processed)
    # normalize
    x_processed = Normalize1D.apply(x_processed, norm_type='min_max')
    return x_processed

In [3]:
# process multi-channel signal
def process_signalnd(x, raw_fs=1000, low_fs=1, high_fs=120, notch_fs=60, Q=20, window_size=250, step_size=50, target_fs=512):
    """
    @param x: signal of a single channel
    @param raw_fs: original sampling rate
    @param low_fs: low cutoff frequency
    @param high_fs: high cutoff frequency
    @param notch_fs: notch cutoff frequency
    @param Q: Q factor
    @param window_size: windows size for detrending
    @param step_size: step size for detrending
    @param target_fs: target sampling rate for resampling step
    """
    num_channels = x.shape[1]
    x_processed = np.array([])
    for i in range(num_channels):
        # process each channel
        channel_processed = process_signal1d(x[:, i], raw_fs, low_fs, high_fs, notch_fs, Q, window_size, step_size, target_fs)
        channel_processed = np.expand_dims(channel_processed, axis=1)
        if i == 0:
            x_processed = channel_processed
            continue
        x_processed = np.hstack((x_processed, channel_processed))
    return x_processed

Load multi-channel signal from file.

In [4]:
# load emg signal from csv file
emg_raw = pd.read_csv('./data/example/emg2.csv')

# print first ten EMG samples
print('EMG samples:\n', emg_raw.head(10))

# print number of samples
print('\nNumber of samples:\n', len(emg_raw))

# print number of channels
print('\nNumber of channels:\n', len(emg_raw.columns))

# get emg raw data
emg_raw = emg_raw.values

EMG samples:
      CH1    CH2    CH3    CH4
0  727.0  725.0  724.0  726.0
1  736.0  733.0  737.0  735.0
2  737.0  735.0  743.0  739.0
3  738.0  738.0  749.0  741.0
4  739.0  739.0  749.0  736.0
5  737.0  735.0  740.0  730.0
6  734.0  730.0  735.0  727.0
7  732.0  729.0  730.0  725.0
8  731.0  725.0  726.0  723.0
9  724.0  720.0  720.0  714.0

Number of samples:
 10000

Number of channels:
 4


Clean raw signal.

In [5]:
# Clean emg raw signal
raw_fs = 1000
target_fs = 512
low_fs = 10
high_fs = 120
notch_fs = 60
Q = 20
windows = 512
steps = 50
emg_processed = process_signalnd(emg_raw, raw_fs, low_fs, high_fs, notch_fs, Q, windows, steps, target_fs)

Segment multi-channel signal.

In [6]:
window_size = 512
step_size = 32
segments = SegmentND.apply(emg_processed, window_size, step_size)

# print number of segments
print('\nNumber of segments:\n', len(segments))


Number of segments:
 144


## Part 1: EMG Feature Extraction

With the aim to facilitate the classification process, the raw signal should be transformed into relevant data structure or feature vector to highlight the important data for pattern recognition. There are three types of features in EMG control system: 1) time domain, 2) frequency domain, 3) time-frequency domain.

In [7]:
# extract feature from one segment of signal
segment = segments[0]

# print shape of segment
# (number of channels, number of samples per channel)
print('Shape (# of channels, channel length): {}'.format(segment.shape))

window_size = 10    # window length to get feature

Shape (# of channels, channel length): (512, 4)


### 1.1. Time Domain Features

The time domain features are the most popular in EMG pattern recognition, because they are easy and quick to calculate as well as they do not require transformation. The time domain features usually are computed based on signal amplitude, and resultant values.

##### Maximum Peak Value (MPK)

It is the maximum absolute vlaue of the signal window.

$$\text{MPK}_{k}(x)=\text{max}{|x_{i}|}$$

where $x_{i}$ is value of signal window $k$

In [8]:
peaks = MaxPeak.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(peaks.shape))

Shape (# of channels, feature length): (52, 4)


##### Mean Value (MN)

When the analysis time is not too short. the mean is essentially a constant. Therefore, any shift in value of the mean are indictive of changes in potential.

$$\text{MN}_{k}(x)=\frac{1}{N}\sum_{i=1}^{N}{x_{i}}$$

where $x_{i}$ is value of signal window $k$, $N$ is the length of the signal window.

In [9]:
means = Mean.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(means.shape))

Shape (# of channels, feature length): (52, 4)


##### Variance (VAR)

It is a statistical feature.

$$\text{VAR}_{k}(x)=\sqrt{\frac{1}{N}\sum_{i=1}^{N}{(x_{i}-\bar{x})^{2}}}$$

where $x_{i}$ is value of signal window $k$, $\bar{x}$ is mean value of signal window, $N$ is the length of the signal window.

In [10]:
vars = Variance.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(vars.shape))

Shape (# of channels, feature length): (52, 4)


##### Standard Deviation (STD)

Is is a statistical feature.

$$\text{STD}_{k}(x)=\sqrt{\frac{\sum_{i=1}{N}{(x_{i} - \bar{x})}^{2}}{N-1}}$$

where $x_{i}$ is value of signal window $k$, $\bar{x}$ is mean value of signal window, $N$ is the length of the signal window.

In [11]:
stds = StandardDeviation.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(stds.shape))

Shape (# of channels, feature length): (52, 4)


##### Skewness (S)

It is statistical feature. It measures the degree of deviation from the symmetry of Normal or Gaussian distribution.

$$\text{S}_{k_{mc}}(x)=\frac{\sum_{i=1}^{N}{\frac{(x_i-\bar{x})^{3}}{N}}}{(\sum_{i=1}^{N}{\frac{(x_{i} - \bar{x})}{N})^{3/2}}$$

where $x_{i}$ is value of signal window $k$, $\bar{x}$ is mean value of signal window, $N$ is the length of signal window, $S_{k_{mc}}$ is the moment coefficient of skewness.

In [12]:
skews = Skewness.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(skews.shape))

Shape (# of channels, feature length): (52, 4)


##### Kurtosis (K)

It is a statistical feature. It measures the peakedness or flatness of a distribution.

$$\text{K}_{k_{mc}}(x)=\frac{\sum_{i=1}{N}{\frac{(x_{i} - \bar{x})^{4}}{N}}}{\sum_{i=1}^{N}(\frac{(x_{i} - \bar{x})^{2}}{N})^{2}} - 3$$

where $x_{i}$ is value of signal window $k$, $\bar{x}$ is mean value of signal window, $N$ is the length of signal window, $K_{k_{mc}}$ is the moment coefficient of kurtosis.

In [13]:
kurts = Kurtosis.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(kurts.shape))

Shape (# of channels, feature length): (52, 4)


##### Root Mean Square (RMS)

It is modeled as amplitude modulated Gaussian random process whose RMS is related to the constant force and non-fatiguing contraction.

$$\text{RMS}_{k}=\sqrt{\frac{1}{N}\sum_{i=1}^{N}{x_{i}^{2}}}$$

where $x_{i}$ is value of signal segment $k$, $N$ is the length of the signal segment.

In [14]:
rmss = RootMeanSquare.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(rmss.shape))

Shape (# of channels, feature length): (52, 4)


##### Waveform Length (WL)

It is the cumulative length of the waveform over the segment. The resultant values indicate a measure of waveform amplitude, frequency and duration all within a single parameter.

$$\text{WL}_{k}=\sum_{i=1}^{N-1}{|x_{i+1}-x_{i}|}$$

where $x_{i}$ and $x_{i+1}$ is consecutive values of signal segment $k$, $N$ is the length of the signal segment.

In [15]:
wls = WaveformLength.apply(segment, window_size)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(wls.shape))

Shape (# of channels, feature length): (52, 4)


##### William Amplitude (WAMP)

It calculates the number of times that the absolute value of the difference between EMG signal amplitude of two consecutive samples ($x_{i}$ and $x_{i+1}$) exceeds a predetermined threshold $\epsilon$.

$$\text{WAMP}_{k}=\sum_{i=1}^{N-1}{f(x_{i}-x_{i+1})}$$

$$f(x)=\begin{cases}{1}&&{x>\epsilon}\\{0}&&{\text{otherwise}\end{cases}$$

where $x_{i}$ and $x_{i+1}$ is consecutive values of signal segment $k$, $N$ is the length of the signal segment, $\epsilon$ is threshold value.

In [16]:
eps = 0.3
wamps = WillisonAmplitude.apply(segment, window_size, eps)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(wamps.shape))

Shape (# of channels, feature length): (52, 4)


### 1.2. Frequency Domain Features

The frequency domain features are based on estimated Power Spectrum Density (PSD) of signal and are computed by Fast Fourier Transforms (FFT). In comparison with time domain features, frequency domain features requires more computation.

##### Power Spectrum Density (PSD)

The feature shows at what point frequency variations are strong and weak.

$$\text{PSD}={\Bigg|\sum_{i=0}^{N-1}{x_{i}e^{\frac{-j2{\pi}ki}{N}}}\Bigg|}^{2}$$

where $k=0,1,...,N-1$, $N$ is the length if the signal data, $x_{i}$ represents the discrete samples of signal data.

##### Frequency Median (FMD)

The frequency median splits the power spectrum density into two equal parts.

$$\text{FMD}=\frac{1}{2}\sum_{i=1}^{N}{\text{PSD}_{i}}$$

where $N$ is the length of the power spectrum density, and $\text{PSD}_{i}$ is the $i^{th}$ line of power spectrum density.

In [17]:
min_fs = 2
fs = 512
window_size = 512
average = 'median'
fmds = PSD.apply(segment, window_size, fs, average)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(fmds.shape))

Shape (# of channels, feature length): (257, 4)


##### Frequency Mean (FMN)

$$\text{FMN}=\frac{\sum_{i=1}^{M}{f_{i}\text{PSD}_{i}}}{\sum_{i=1}^{M}{\text{PSD}_{i}}}$$

where $N$ is the length of the power spectrum density, $f_{i}=(i\times\text{sampling_rate})$ and $\text{PSD}_{i}$ is the $i^{th}$ line of power spectrum density.

In [18]:
fs = 512
window_size = 512
average = 'mean'
fmns = PSD.apply(segment, window_size, fs, average)
# print feature shape
print('Shape (# of channels, feature length): {}'.format(fmns.shape))

Shape (# of channels, feature length): (257, 4)


### 1.3. Time-Frequency domain features

Time-frequency domain representation can localize the energy of the signal both in time and in frequency, allowing a more accurate description of the physical phenomenon, but these features generally requires a transformation that could be computationally heavy.

##### Short TIme Fourier Transform (STFT).

As the extension of Fourier transform method by devising the input signal into segments, by doing each signal in the each window can be assumed to be stationary.

$$\text{STFT}_{x}(t, w)=\int{W^{*}(\tau-t)x(\tau)e^{-jw\tau}}d\tau$$

where $W$ is the window function, $*$ is complex conjugate, $\tau$ represents time, $w$ stands for frequency.

In [19]:
stfts = STFT.apply(segment,np.arange(1, 128))
# return stfts
# print feature shape
print('Shape (# of channels, frequency, stft per frequency): {}'.format(stfts.shape))

Shape (# of channels, frequency, stft per frequency): (64, 49, 4)


##### Wavelet Transform (WT)

It is a transform where a signal is integrated with a shifted and scaled mother wavelet function.

$$\text{WT}_{x}(a,b)=\int{{x(t)}{(\frac{1}{\sqrt{a}})\Psi^{*}{(\frac{t-b}{a})}}dt}$$

where $x(t)$ is the input signal, $\Psi^{*}$ is the complex conjugate of the mother wavelet transform, $\Psi(\frac{t-b}{a})$ is the shifted and scaled version of the wavelet at time $b$ and scale $a$.

In [20]:
window_size = 128
widths = 50
# cwts = CWT.apply(segment, window_size, widths)
L = segment.shape[0]
num_channels = segment.shape[1]
if isinstance(widths, int):
    widths = np.arange(1, widths + 1)
cwts = np.array([])
for i in range(num_channels):
    cwt = signal.cwt(segment[:, i], signal.ricker, widths, dtype=np.float64)
    if i == 0:
        cwts = cwt
        continue
    cwts = np.dstack((cwts, cwt))
# print feature shape
print('Shape (# of channels, # of scale, cwt per scale): {}'.format(cwts.shape))

Shape (# of channels, # of scale, cwt per scale): (50, 512, 4)


## Part 3: Practices

### Practice 1:

Try to change some parameters from feature extraction functions and find the differences.

### Practice 2:

Try to extract the features list of signal segments. Save the features into a list.